<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/KRX_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import json
import requests
import pandas as pd
from io import BytesIO
import pytz
from datetime import datetime, date, timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/[Shared]K-Project/full_code.pkl',"rb") as f:  # k-프로젝트 공유드라이브로 연결.
    full_code = pickle.load(f)
full_code

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2580,KR7000542001,000547,흥국화재2우B,KOSPI,상장
2581,KR7000541003,000545,흥국화재우,KOSPI,상장
2582,KR7003280005,003280,흥아해운,KOSPI,상장
2583,KR7037440005,037440,희림,KOSDAQ,상장


In [ ]:
def today_date():   #일주일 날짜 구하기.
  tz = pytz.timezone('Asia/Seoul')  #한국시간 맞추기.
  raw_dates = datetime.now(tz)
  day7_raw = raw_dates - timedelta(7) 
  day7 =  day7_raw.strftime("%Y%m%d")  # 일주일전 문자열로 변환
  today = raw_dates.strftime("%Y%m%d")  # 오늘 날짜 문자열로 변환
  return day7, today




def week_delisting (day7,today):     #  상장 폐지 종목 불러오기 (KRX_상장폐지종목현황_화면번호(20037))

  headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36,'}  

  url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

  data = {'bld': 'dbms/MDC/STAT/issue/MDCSTAT23801',
        'mktId': 'ALL',
        'tboxisuCd_finder_listdelisu0_0':'전체',
        'isuCd': 'ALL',
        'isuCd2': 'ALL',
        'codeNmisuCd_finder_listdelisu0_1':'',
        'param1isuCd_finder_listdelisu0_1':'',
        'strtDd': day7,
        'endDd': today, 
        'share': '1', 
        'csvxls_isNo': 'true',}

  r = requests.post(url,data,headers=headers)
  j = json.loads(r.text)

  df = pd.json_normalize(j['output'])
  return df[['ISU_CD','ISU_NM','MKT_NM','DELIST_DD','DELIST_RSN_DSC']] #종목코드, 종목명, 시장구분,증권구분, 폐지일, 폐지사유 열만 추출
 




def issues_for_administration():  # 관리종목 현황 종목 (krx화면번호_ 20012)
  headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36,'} 

  data ={'bld': 'dbms/MDC/STAT/issue/MDCSTAT21401',
        'mktId': 'ALL',
        'csvxls_isNo': 'true',}

  url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

  r = requests.post(url,data,headers=headers)
  j = json.loads(r.text)

  df = pd.json_normalize(j['output'])
  
  return df[['ISU_CD','ISU_NM','MKT_NM','FST_DESIGN_DD','LIST_BZ_RSN_NM']] #종목코드,종목명, 시장구분, 최초지정일, 지정사유 열만 추출.



def KRX_search_fullcode(new_code):   # 신규상장  full_code krx에서 가져오기.
  headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36,'}  

  data = {'mktsel': 'ALL',
        'typeNo': '0',
        'searchText': new_code,
        'bld': 'dbms/comm/finder/finder_stkisu',}

  url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

  r = requests.post(url,data,headers=headers)
  j = json.loads(r.text)

  df = pd.json_normalize(j['block1'])
  return df[['full_code','short_code','codeName','marketEngName']]



def get_fullcode(df,short_code) :     # 구축된 데이터프레임에서 종목코드로 full_code 찾기
  df= df.set_index('short_code') #종목코드 index 설정
  full_code = df.loc[short_code]['full_code']
  return full_code



# 신규상장기업현황 ( https://kind.krx.co.kr/listinvstg/listingcompany.do?method=searchListingTypeMain) 가져오기.
def new_listing(today): 

  headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36,'} 

  data = {'method': 'searchListingTypeSub',
        'currentPageSize': '15',
        'pageIndex': '1',
        'orderMode': '1',
        'orderStat': 'D',
        'repIsuSrtCd': '',
        'isurCd': '',
        'forward': 'listingtype_sub',
        'listTypeArrStr': '01|02|03|04|05|',
        'choicTypeArrStr':'' ,
        'searchCodeType': '',
        'searchCorpName': '',
        'secuGrpArrStr': '0|ST|FS|MF|SC|RT|DR|',
        'marketType': '',
        'searchCorpNameTmp':'' ,
        'country': '410',
        'industry': '',
        'repMajAgntDesignAdvserComp': '',
        'repMajAgntComp': '',
        'designAdvserComp': '',
        'secuGrpArr': '0',
        'secuGrpArr': 'ST|FS',
        'secuGrpArr': 'MF|SC|RT',
        'secuGrpArr': 'DR',
        'listTypeArr': '01',
        'listTypeArr': '02',
        'listTypeArr': '03',
        'listTypeArr': '04',
        'listTypeArr': '05',
        'fromDate': today,
        'toDate': today,}

  url = "https://kind.krx.co.kr/listinvstg/listingcompany.do"

  r = requests.post(url,data,headers=headers)
  f = BytesIO(r.content)
  dfs = pd.read_html(f, encoding='UTF8' )
  df = dfs[0].copy()
  df = df.iloc[:,[0,1,2]]

  df =df.rename(columns = {'Unnamed: 0':'회사명','Unnamed: 1':'상장일','Unnamed: 2':'상장유형'})
  return df



In [ ]:
# 당일 상장폐지항목 df에서 상장폐지 상태로 변경하기.
def change_delisting(df, week_delisting,today): 

  for i in range (week_delisting.shape[0]) : 

    check = week_delisting.iloc[i,3] == today
    if (check):   #상장폐지 항목들이 오늘일 때만.
      len_shortcode = len(week_delisting.iloc[i,0])

      if (len_shortcode == 6) :  #종목코드 숫자가 6개일때만 (수익증권 같은 상장폐지항목은 저장된 df에 없기 때문에)
        index = df[df.codeName == week_delisting.iloc[i,1]].index
        df.iloc[index,4] = '상장폐지' 

  return df

In [ ]:
# 최종 full_code 업데이트 함수.
def update_fullcode(df):

  day7, today = today_date()  #일주일 기간 날짜 구하기.

  #임시 확인용.
  print('일주일 전 : '+ day7)
  print('오늘 : '+ today+'\n') 

  print('한 주 상장폐지 종목 조회')
  week_delisting01 = week_delisting(day7,today) #한 주동안 상장폐지된 항목 조회
  week_delisting01['DELIST_DD'] = week_delisting01['DELIST_DD'].str.replace('/','')

  #임시 확인용.
  print(week_delisting01+ '\n') 
  print(week_delisting01['DELIST_DD'])  

  df = change_delisting(df, week_delisting01, today) # 상장폐지 항목 df에서 상장폐지 상태 변경. 

  # 신규상장 df에 추가하기.

  return df

In [ ]:
fullcode_copy = full_code.copy()
fullcode_copy

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2580,KR7000542001,000547,흥국화재2우B,KOSPI,상장
2581,KR7000541003,000545,흥국화재우,KOSPI,상장
2582,KR7003280005,003280,흥아해운,KOSPI,상장
2583,KR7037440005,037440,희림,KOSDAQ,상장


In [ ]:
fullcode_copy = update_fullcode(fullcode_copy)
check = fullcode_copy == full_code
print('\n <잘 작동되는 지 확인>')
check.sum()

일주일 전 : 20210926
오늘 : 20211003

한 주 상장폐지 종목 조회
       ISU_CD  ...                                     DELIST_RSN_DSC
0  2177301B\n  ...                                                 \n
1    111820\n  ...  기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 ...
2  74701772\n  ...                        수익증권 신탁기간의 만료 또는 전액 중도 환수\n
3  74702772\n  ...                        수익증권 신탁기간의 만료 또는 전액 중도 환수\n

[4 rows x 5 columns]
0    20211001
1    20210930
2    20210929
3    20210929
Name: DELIST_DD, dtype: object

 <잘 작동되는 지 확인>


full_code        2585
short_code       2585
codeName         2585
marketEngName    2585
status           2585
dtype: int64

<br><br>
<hr>
<h2>함수 작동 확인용</h2>

In [ ]:
day7, today = today_date() #일주일 기간 날짜 구하기.
print(day7, today)

20210926 20211003


In [ ]:
week_delisting = week_delisting(day7,today) #한 주동안 상장폐지된 항목 조회
week_delisting

,ISU_CD,ISU_NM,MKT_NM,DELIST_DD,DELIST_RSN_DSC
0,2177301B,강스템바이오텍 23R,KOSDAQ,2021/10/01,
1,111820,지유온,KOSDAQ,2021/09/30,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정
2,74701772,이지스코어부동산제117호A,KOSPI,2021/09/29,수익증권 신탁기간의 만료 또는 전액 중도 환수
3,74702772,이지스코어부동산제117호F,KOSPI,2021/09/29,수익증권 신탁기간의 만료 또는 전액 중도 환수


- 지금까지 당일 상장폐지 항목들만  추출 했었는데<br>  상장폐지가 없는 날에는 에러가 발생하여 일주일동안 상장폐지한 항목들을 추출하고<br>
 당일 상장폐지 항목을 걸러내는 방식으로 바뀌었습니다. 

In [ ]:
week_delisting['DELIST_DD'] = week_delisting['DELIST_DD'].str.replace('/','')
week_delisting

,ISU_CD,ISU_NM,MKT_NM,DELIST_DD,DELIST_RSN_DSC
0,2177301B,강스템바이오텍 23R,KOSDAQ,20211001,
1,111820,지유온,KOSDAQ,20210930,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정
2,74701772,이지스코어부동산제117호A,KOSPI,20210929,수익증권 신탁기간의 만료 또는 전액 중도 환수
3,74702772,이지스코어부동산제117호F,KOSPI,20210929,수익증권 신탁기간의 만료 또는 전액 중도 환수


In [ ]:
print(week_delisting.iloc[0,3] == '20211001')
print(week_delisting.iloc[0,3] == today) #오늘 10월3일.

True
False


In [ ]:
week_delisting.shape[0]

4

In [ ]:
issues_for_administration() #관리종목 검색.

,ISU_CD,ISU_NM,MKT_NM,FST_DESIGN_DD,LIST_BZ_RSN_NM
0,131100,스카이이앤엠,KOSDAQ,2021/09/23,최근3사업연도중 2사업연도 자기자본 50%초과 법인세비용차감전계속사업손실 발생
1,317030,케이비17호스팩,KOSDAQ,2021/09/23,SPAC 상장예비심사청구서 미제출 등
2,148140,비디아이,KOSDAQ,2021/08/18,"반기검토(감사)의견 부적정, 의견거절 또는 범위제한 한정"
3,221610,자안바이오,KOSDAQ,2021/08/18,"자기자본 10억원 미만,상장폐지사유 발생,자본잠식률 50% 이상,반기검토(감사)의견..."
4,078650,지나인제약,KOSDAQ,2021/08/18,"반기검토(감사)의견 부적정, 의견거절 또는 범위제한 한정,자본잠식률 50% 이상"
...,...,...,...,...,...
108,079190,EMW,KOSDAQ,2018/10/17,상장적격성 실질심사 대상 결정
109,065150,엠피대산,KOSDAQ,2018/08/16,최근 4사업연도 연속 영업손실 발생
110,050320,아래스,KOSDAQ,2017/03/23,"상장폐지사유 발생,반기검토(감사)의견 부적정, 의견거절 또는 범위제한 한정,최근 4..."
111,115530,씨엔플러스,KOSDAQ,2017/03/16,"상장적격성 실질심사 대상 결정,사업보고서 미제출,상장폐지사유 발생"


In [ ]:
new_listing = new_listing(today) #오늘은 새로 상장된 주식이 없음.
new_listing

,회사명,상장일,상장유형
0,조회된 결과값이 없습니다.,조회된 결과값이 없습니다.,조회된 결과값이 없습니다.


In [ ]:
new_listing.iloc[0,0] == '조회된 결과값이 없습니다.'

True

In [ ]:
def change_new_listing (df):  #새로 상장된 주식 full_code에 추가하기
  result =  new_listing.iloc[0,0] != '조회된 결과값이 없습니다.'   
  if (result):  #신규 상장이 있으면.
    #
  return df

In [ ]:
#공유 드라이브에 데이터 저장.
with open('/content/drive/MyDrive/[Shared]K-Project/full_code.pkl', 'wb') as f:
	pickle.dump(full_code, f)